# Redes neuronales

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.0


## Conceptos

| Concepto      | Qué significa                             |
| ------------- | ----------------------------------------- |
| Forward pass  | Calcular predicción usando pesos actuales |
| Loss          | Cuánto se equivoca el modelo              |
| Backward pass | Calcula derivadas con `GradientTape`      |
| Gradient      | Dirección de corrección para cada peso    |
| Update rule   | `w = w - learning_rate * gradient`        |
| Objetivo      | Minimizar la pérdida                      |

- Recordatorio del flujo

| Paso | Nombre         | Qué pasa                                                       |
| ---- | -------------- | -------------------------------------------------------------- |
| 1  | Forward pass   | La red hace predicción con los pesos actuales                  |
| 2  | Compute loss   | Comparas predicción vs valor real                              |
| 3  | Backward pass  | Calculas gradientes (derivadas) con `GradientTape`             |
| 4  | Update weights | Ajustas pesos usando gradientes (SGD o regla de actualización) |


## Ejemplo 1 — Una neurona simple (y = wx + b)

- Objetivo: aprender el valor correcto de w y b para aproximar la función:
    - y = 2x + 1

In [5]:
# Datos
x = tf.constant([1.0, 2.0, 3.0, 4.0])
y_real = tf.constant([3.0, 5.0, 7.0, 9.0])  # 2x + 1

print("Datos de entrada (x):", x.numpy())
print("Datos reales (y_real):", y_real.numpy())

Datos de entrada (x): [1. 2. 3. 4.]
Datos reales (y_real): [3. 5. 7. 9.]


In [6]:
# Pesos iniciales (random)
w = tf.Variable(tf.random.normal([1], dtype=tf.float32))
b = tf.Variable(tf.random.normal([1], dtype=tf.float32))

lr = 0.1  # Learning rate

print("Pesos iniciales: w =", w.numpy(), ", b =", b.numpy())
print("Learning rate:", lr)

Pesos iniciales: w = [1.3552873] , b = [0.00364656]
Learning rate: 0.1


In [7]:
# Entrenamiento

# Número de épocas
epochs = 50

for epoch in range(epochs+1):
    with tf.GradientTape() as tape:
        # Forward pass, calcular predicción
        y_pred = w * x + b

        # Loss: MSE, calcular pérdida
        loss = tf.reduce_mean((y_real - y_pred) ** 2)

    # Backward pass, calcular gradientes
    # En qué dirección y cuánto deben cambiar w y b para disminuir la pérdida.
    # Matemáticamente son derivadas parciales
    # Son como señales de corrección
    dw, db = tape.gradient(loss, [w, b])

    # Update, actualizar pesos de acuerdo a los gradientes
    # lr = learning rate: qué tan rápido aprende
    w.assign_sub(lr * dw)
    b.assign_sub(lr * db)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}  Loss={loss.numpy():.4f}  w={w.numpy()}  b={b.numpy()}")

Epoch 0  Loss=7.3219  w=[2.8205333]  b=[0.5252736]
Epoch 10  Loss=0.0465  w=[2.1803463]  b=[0.50511414]
Epoch 20  Loss=0.0240  w=[2.125319]  b=[0.63219166]
Epoch 30  Loss=0.0131  w=[2.09233]  b=[0.72855055]
Epoch 40  Loss=0.0071  w=[2.0681264]  b=[0.79969984]
Epoch 50  Loss=0.0039  w=[2.0502696]  b=[0.8522009]


- Resumen del flujo

| Paso           | Qué hace                            | Metáfora                                   |
| -------------- | ----------------------------------- | ------------------------------------------ |
| 1 Forward    | Predice algo con `w` y `b` actuales | El estudiante responde una pregunta        |
| 2 Loss       | Mide qué tan mal respondió          | Le dices qué tan equivocado estuvo         |
| 3 Gradientes | Calcula cómo corregir               | Le dices **cómo mejorar**, no la respuesta |
| 4 Update     | Cambia `w` y `b`                    | Aprende y ajusta su conocimiento           |

- Resumen:
    - w y b: lo que el modelo sabe (parametros entrenables)
    - y_pred = w * x + b: predicción actual
    - loss: qué tanto se equivocó
    - dw, db: cómo debe cambiar
    - assign_sub: aplica el aprendizaje

- Metáfora rápida. Imagina que quieres lanzar una flecha a un blanco con los ojos vendados.
    - Primer tiro → cae lejos (loss alto)
    - Te dicen: "más arriba, más derecha" (gradiente)
    - Ajustas el tiro (updates)
    - Repetición → mejora progresiva (epochs)
    - Al final → tiro perfecto (loss casi cero)
- Eso es gradient descent.

## Ejemplo 2 — Neurona con activación (sigmoid)

- Ahora hacemos una neurona que aprenda una puerta lógica AND:

| x1 | x2 | y |
| -- | -- | - |
| 0  | 0  | 0 |
| 0  | 1  | 0 |
| 1  | 0  | 0 |
| 1  | 1  | 1 |

- Una función de activación es la transformación aplicada al valor z para convertirlo en una salida útil.
- Con activación, la red puede aprender:
    - límites no lineales
    - clasificaciones
    - patrones complejos
    - lógica (como AND, OR)

In [8]:
# Dataset
X = tf.constant([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y_true = tf.constant([[0.],[0.],[0.],[1.]])

print("\nDataset (X):")
print(X.numpy())
print("Etiquetas reales (y_true):")
print(y_true.numpy())

# Pesos
w = tf.Variable(tf.random.normal([2,1]))
b = tf.Variable(tf.zeros([1]))

lr = 0.1

print("\nPesos iniciales para clasificación:\n w =", w.numpy(), ", b =", b.numpy())
print("Learning rate:", lr)


Dataset (X):
[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Etiquetas reales (y_true):
[[0.]
 [0.]
 [0.]
 [1.]]

Pesos iniciales para clasificación:
 w = [[ 1.0883665 ]
 [-0.92451656]] , b = [0.]
Learning rate: 0.1


In [9]:
for epoch in range(2000):
    with tf.GradientTape() as tape:
        # Forward
        z = tf.matmul(X, w) + b
        y_pred = tf.sigmoid(z)

        # Loss binary crossentropy
        loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))

    # Backpropagation
    grads = tape.gradient(loss, [w, b])

    # Update
    w.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])

    if epoch % 400 == 0:
        print(f"Epoch {epoch} Loss={loss.numpy():.5f}")
        print(" w =", w.numpy(), ", b =", b.numpy())

print("\nPredicciones finales:")
print(tf.round(tf.sigmoid(tf.matmul(X, w) + b)))

Epoch 0 Loss=0.75512
 w = [[ 1.0811429]
 [-0.9201393]] , b = [-0.02682458]
Epoch 400 Loss=0.25765
 w = [[1.9059958]
 [1.6442373]] , b = [-2.9538276]
Epoch 800 Loss=0.16732
 w = [[2.7332985]
 [2.680386 ]] , b = [-4.2896504]
Epoch 1200 Loss=0.12416
 w = [[3.3561697]
 [3.3413727]] , b = [-5.2316513]
Epoch 1600 Loss=0.09843
 w = [[3.845845]
 [3.840657]] , b = [-5.962949]

Predicciones finales:
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [1.]], shape=(4, 1), dtype=float32)


### Glosario Esencial de Redes Neuronales

### 1. Entrada (Input)

Datos que alimentan la red.
Puede ser:

* Un número (ej: temperatura)
* Un vector (ej: pixeles de un dígito)
* Una matriz (imagen en escala de grises)
* Un tensor (imagen con canales o batch)

Ejemplo:
Para reconocer si una imagen contiene un gato, los valores de los píxeles son la entrada.

### 2. Neurona (Nodo)

Es la unidad básica de cálculo.
Toma entradas, las multiplica por pesos, suma el bias, aplica una activación y produce una salida.

La fórmula base es:

```bash
z = w1*x1 + w2*x2 + ... + wn*xn + b
y = activation(z)
```

Igual que una neurona biológica: recibe señales, las procesa y dispara si son suficientes.

### 3. Pesos (Weights, w)

Son los valores que dicen qué tan importante es cada entrada.

Ejemplo:

* Si `w1 = 0`, la entrada x1 no importa.
* Si `w2 = 100`, esa entrada domina la decisión.

Los pesos son el conocimiento aprendido.

### 4. Bias (b)

Número libre que permite desplazar la salida antes de aplicar la activación.
Actúa como un umbral: decide cuánta evidencia hace falta para activar la neurona.

Sin bias, todas las decisiones pasarían por el punto (0,0) → menos flexible.

### 5. Función de Activación

Transforma el valor `z` en algo útil.
Convierte una suma lineal en una decisión no lineal, permitiendo aprender patrones complejos.

Ejemplos:

| Activación | Uso                   | Ejemplo salida                       |
| ---------- | --------------------- | ------------------------------------ |
| Sigmoid    | Clasificación binaria | 0.01 → 0.99                          |
| ReLU       | Redes profundas       | max(0, z)                            |
| Softmax    | Multiclase            | probabilidades (clase 0.2, 0.5, 0.3) |

Sin activación, una red neuronal sería solo una regresión lineal.

### 6. Capa (Layer)

Una colección de neuronas trabajando juntas.

Tipos comunes:

* Capa densa (Fully Connected): cada neurona conecta con todas las entradas.
* Convolucional (CNN): detecta patrones espaciales (imágenes).
* Recurrente (RNN, LSTM, GRU): maneja secuencias (texto, audio).
* Transformers: atención, NLP moderno (ChatGPT, BERT).

### 7. Forward Pass (Propagación hacia adelante)

El proceso donde la red toma los datos y genera una predicción usando los pesos actuales.

Es como un estudiante respondiendo un examen sin mirar la respuesta aún.

### 8. Función de Pérdida (Loss / Cost Function)

Mide qué tan mal predijo la red comparado con el objetivo real.

Ejemplos:

* MSE (Regresión): `(y_true - y_pred)²`
* Binary Crossentropy (Binario)
* Categorical Crossentropy (Multiclase)

Si el loss baja → la red está aprendiendo.

### 9. Gradient (Gradiente)

La derivada de la pérdida respecto a los pesos.

Dicen:

> "Si ajustas este peso en sentido positivo o negativo, ¿la pérdida mejora?"

Son instrucciones, no los pesos finales.

### 10. Backpropagation

El algoritmo que usa los gradientes para ajustar los pesos calculando derivadas desde la salida hacia atrás.

"Aprender de los errores."

### 11. Optimizador (Optimizer)

Método matemático que usa los gradientes para actualizar los pesos.

Ejemplos:

| Optimizador | Característica                |
| ----------- | ----------------------------- |
| SGD         | Simple, lento                 |
| Momentum    | Más estable                   |
| Adam        | Más usado, rápido, adaptativo |
| RMSProp     | Bueno para secuencias         |

Decide cómo corregir los pesos.

### 12. Learning Rate (LR)

Controla qué tan grande es cada paso de actualización.

* Muy bajo → aprendizaje lento.
* Muy alto → aprendizaje inestable.

Es la “velocidad de aprendizaje”.

### 13. Epoch

Una pasada completa sobre el dataset durante entrenamiento.

Si tienes 100 ejemplos y batch size 10 → una época tiene 10 actualizaciones.

### 14. Batch y Batch Size

* Batch: subconjunto de datos usados en una actualización.
* Batch size: tamaño de ese subconjunto.

Entrenar por batches evita usar toda la memoria y hace más eficiente el aprendizaje.

### 15. Predicción / Output

El resultado final que produce la red.

* Puede ser un número (regresión)
* Probabilidad (clasificación)
* Clase (argmax)

### MINI RESUMEN TABLA

| Elemento     | Rol                         |
| ------------ | --------------------------- |
| Pesos        | Aprendizaje principal       |
| Bias         | Ajusta el umbral            |
| Activación   | Hace no lineal la función   |
| Forward pass | Predice                     |
| Loss         | Evalúa qué tan mal          |
| Gradientes   | Señalan corrección          |
| Optimizador  | Ajusta pesos                |
| Epoch        | Iteración sobre dataset     |
| Batch        | División para entrenamiento |

### En una frase:

> Una red neuronal es un conjunto de neuronas conectadas, con pesos y bias ajustados mediante gradientes durante muchos epochs usando una función de pérdida y un optimizador, donde las activaciones permiten aprender patrones complejos.


## Ejemplo Final: Clasificador binario de puntos en 2D

- Queremos que la red clasifique puntos en:
    - Clase 0 → izquierda de la línea
    - Clase 1 → derecha de la línea

| x1  | x2  | Clase |
| --- | --- | ----- |
| 1.0 | 1.0 | 0     |
| 2.0 | 1.0 | 0     |
| 3.0 | 2.0 | 0     |
| 4.0 | 5.0 | 1     |
| 5.0 | 7.0 | 1     |
| 6.0 | 9.0 | 1     |


In [10]:
# 1) DATASET
X = tf.constant([
    [1.,1.],
    [2.,1.],
    [3.,2.],
    [4.,5.],
    [5.,7.],
    [6.,9.]
])

y_true = tf.constant([
    [0.],
    [0.],
    [0.],
    [1.],
    [1.],
    [1.]
])

print("Dataset (X):")
print(X.numpy())
print("Etiquetas reales (y_true):")
print(y_true.numpy())

Dataset (X):
[[1. 1.]
 [2. 1.]
 [3. 2.]
 [4. 5.]
 [5. 7.]
 [6. 9.]]
Etiquetas reales (y_true):
[[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [11]:
# 2) MODEL PARAMETERS (learnable)
W = tf.Variable(tf.random.normal([2, 1]))  # weights
b = tf.Variable(tf.zeros([1]))             # bias

# 3) HYPERPARAMETERS
lr = 0.05      # learning rate
epochs = 2000

print("\nPesos iniciales:\n W =", W.numpy(), ", b =", b.numpy())
print("Learning rate:", lr)


Pesos iniciales:
 W = [[-0.04149994]
 [ 1.2491224 ]] , b = [0.]
Learning rate: 0.05


In [12]:
# TRAINING LOOP
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        # Forward pass
        z = tf.matmul(X, W) + b
        y_pred = tf.sigmoid(z)
        
        # Loss (binary crossentropy)
        loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))
    
    # Backpropagation
    gradients = tape.gradient(loss, [W, b])
    
    # Update
    W.assign_sub(lr * gradients[0])
    b.assign_sub(lr * gradients[1])
    
    # Monitor progress occasionally
    if epoch % 400 == 0:
        print(f"Epoch {epoch}, Loss={loss.numpy():.4f}")
        print("Weights:", W.numpy(), "\nBias:", b.numpy(), "\n")

Epoch 0, Loss=0.8953
Weights: [[-0.08340788]
 [ 1.2212142 ]] 
Bias: [-0.02037188] 

Epoch 400, Loss=0.0954
Weights: [[-1.4385846]
 [ 1.8880917]] 
Bias: [-1.6622312] 

Epoch 800, Loss=0.0561
Weights: [[-1.7291387]
 [ 2.3555784]] 
Bias: [-2.3420618] 

Epoch 1200, Loss=0.0398
Weights: [[-1.9013381]
 [ 2.651195 ]] 
Bias: [-2.7959914] 

Epoch 1600, Loss=0.0308
Weights: [[-2.022794 ]
 [ 2.8673017]] 
Bias: [-3.138076] 



In [13]:
# FINAL PREDICTIONS
print("\n Final Predictions:")
print(tf.round(tf.sigmoid(tf.matmul(X, W) + b)))
print("\nFinal Weights:", W.numpy())
print("Final Bias:", b.numpy())


 Final Predictions:
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]], shape=(6, 1), dtype=float32)

Final Weights: [[-2.1160235]
 [ 3.0370913]]
Final Bias: [-3.41217]


- Interpretación de resultados

| Concepto                   | Lo que sucedió                                         |
| -------------------------- | ------------------------------------------------------ |
| **Forward pass**           | La red hacía predicciones usando `W` y `b`.            |
| **Loss**                   | Medía qué tan lejos estaba de la respuesta real.       |
| **Gradients**              | La red calculó en qué dirección corregir los pesos.    |
| **Optimizer (SGD manual)** | Usó `assign_sub()` para ajustar los parámetros.        |
| **Sigmoid**                | Convirtió `z` en probabilidades entre 0 y 1.           |
| **Epochs**                 | Repeticiones del ciclo mejora→evaluación.              |
| **Resultado final**        | El modelo aprendió a separar las clases correctamente. |

- Significado a nivel conceptual

| Componente     | Rol aprendido                                                                              |
| -------------- | ------------------------------------------------------------------------------------------ |
| `W[0] = -1.73` | Le dice a la red que cuando x1 aumenta, la probabilidad debería bajar                      |
| `W[1] = +2.79` | Le dice que x2 tiene más influencia en la clasificación                                    |
| `b = -3.79`    | Exige que la combinación de entradas sea suficientemente grande antes de activar la salida |

